BERT Tasks
- Masked LM
- Next sentence prediction (eg of text classification)

In [4]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [7]:
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
from transformers import DataCollatorWithPadding

In [8]:
dataset_dict = load_dataset("kri6521/dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [9]:
model_path = "google-bert/bert-base-uncased"

# model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# model
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe":0, "Not Safe":1}
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# freeze all base model parameters
for name, param in model.base_model.named_parameters():  # model obj has base_model attribute and this attribute has named_parameters method
    param.requires_grad = False
# after running this the base model parameters are frozen, only top are trainable

# unfreeze base model pooling layers
for name, param in model.base_model.pooler.named_parameters():
    if "pooler" in name:
        param.requires_grad = True
# we freeze all the model parameters except the last 4 layers

data pre-processing

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [15]:
# to ensure all the examples in a batch have a uniform length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

define evaluation metrics

In [16]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")


# predictions will be between -1 and 1 while labels will be 0 or 1
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  probabilities = np.exp(predictions)/np.exp(predictions).sum(-1, keepdims=True)
  positive_class_;robs = probabilities[:,1]
  auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,references=labels)['roc_auc'],3)
  predicted_classes = np.argmax(predictions, axis=1)
  acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
  return {"accuracy": acc, "auc": auc}

In [18]:
# hyperparameters
lr = 2e-4
batch_size=8
num_epochs=10

training_args = TrainingArguments(
    output_dir="bert-phising-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",  # how often trainer will print training loss
    eval_strategy="epoch",    # how often trainer will print evaluation metrics
    save_strategy="epoch",   # model get saved at every epoch
    load_best_model_at_end=True
)

Fine-tune Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

# accuracy : 0.889, AUC : 0.946